[View in Colaboratory](https://colab.research.google.com/github/ShilpaSangappa/mlblr-EIP2/blob/master/Yolo_Basic.ipynb)

# Session 3 Assignment B
The skip_connection layer has 256 feature maps, to which x20 layer output feature maps will be 
 concatened. 
 Hence, changing x20 output feature maps from 1024 to 256.
 
 epochs changed from 300 to 30. After 9 hours, it was still at epoch5. Stopping there.

In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 30
l = 10
num_filter = 20

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [27]:
input = Input(shape=(img_height, img_width, channel,))

# Layer 1
x01 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
x01 = BatchNormalization(name='norm_1')(x01)
x01 = LeakyReLU(alpha=0.1)(x01)
x01 = MaxPooling2D(pool_size=(2, 2))(x01)

# Layer 2
x02 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(x01)
x02 = BatchNormalization(name='norm_2')(x02)
x02 = LeakyReLU(alpha=0.1)(x02)
x02 = MaxPooling2D(pool_size=(2, 2))(x02)

# Layer 3
x03 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(x02)
x03 = BatchNormalization(name='norm_3')(x03)
x03 = LeakyReLU(alpha=0.1)(x03)

# Layer 4
x04 = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(x03)
x04 = BatchNormalization(name='norm_4')(x03)
x04 = LeakyReLU(alpha=0.1)(x03)

# Layer 5
x05 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(x04)
x05 = BatchNormalization(name='norm_5')(x05)
x05 = LeakyReLU(alpha=0.1)(x05)
x05 = MaxPooling2D(pool_size=(2, 2))(x05)

# Layer 6
x06 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(x05)
x06 = BatchNormalization(name='norm_6')(x06)
x06 = LeakyReLU(alpha=0.1)(x06)

# Layer 7
x07 = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)(x06)
x07 = BatchNormalization(name='norm_7')(x07)
x07 = LeakyReLU(alpha=0.1)(x07)

# Layer 8
x08 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(x07)
x08 = BatchNormalization(name='norm_8')(x08)
x08 = LeakyReLU(alpha=0.1)(x08)
x08 = MaxPooling2D(pool_size=(2, 2))(x08)

# Layer 9
x09 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(x08)
x09 = BatchNormalization(name='norm_9')(x09)
x09 = LeakyReLU(alpha=0.1)(x09)

# Layer 10
x10 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(x09)
x10 = BatchNormalization(name='norm_10')(x10)
x10 = LeakyReLU(alpha=0.1)(x10)

# Layer 11
x11 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(x10)
x11 = BatchNormalization(name='norm_11')(x11)
x11 = LeakyReLU(alpha=0.1)(x11)

# Layer 12
x12 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)(x11)
x12 = BatchNormalization(name='norm_12')(x12)
x12 = LeakyReLU(alpha=0.1)(x12)

skip_connection = x12

# Layer 13
x13 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(x12)
x13 = BatchNormalization(name='norm_13')(x13)
x13 = LeakyReLU(alpha=0.1)(x13)

#skip_connection = x13

x13_maxPool = MaxPooling2D(pool_size=(2, 2))(x13)

# Layer 14
x14 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(x13_maxPool)
x14 = BatchNormalization(name='norm_14')(x14)
x14 = LeakyReLU(alpha=0.1)(x14)

# Layer 15
x15 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(x14)
x15 = BatchNormalization(name='norm_15')(x15)
x15 = LeakyReLU(alpha=0.1)(x15)

# Layer 16
x16 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(x15)
x16 = BatchNormalization(name='norm_16')(x16)
x16 = LeakyReLU(alpha=0.1)(x16)

# Layer 17
x17 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(x16)
x17 = BatchNormalization(name='norm_17')(x17)
x17 = LeakyReLU(alpha=0.1)(x17)

# Layer 18
x18 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(x17)
x18 = BatchNormalization(name='norm_18')(x18)
x18 = LeakyReLU(alpha=0.1)(x18)

# Layer 19
x19 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(x18)
x19 = BatchNormalization(name='norm_19')(x19)
x19 = LeakyReLU(alpha=0.1)(x19)

# Layer 20
#x20 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(x19)
# The skip_connection layer has 256 feature maps, to which x20 layer output feature maps will be 
# concatened. Hence, changing x20 output feature maps to 256.
x20 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(x19)
x20 = BatchNormalization(name='norm_20')(x20)
x20 = LeakyReLU(alpha=0.1)(x20)

#x20_1 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(x20)
#x20_1 = BatchNormalization(name='norm_20')(x20_1)
#x20_1 = LeakyReLU(alpha=0.1)(x20_1)


# Layer 21
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='norm_21')(skip_connection)
skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)
print(skip_connection.get_shape())
print(x20.get_shape())
x21 = concatenate([skip_connection, x20])

# Layer 22
x22 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(x21)
x22 = BatchNormalization(name='norm_22')(x22)
x22 = LeakyReLU(alpha=0.1)(x22)

# Layer 23
x23 = Flatten()(x22)

output = Dense(num_classes, activation='softmax')(x23)



(?, 1, 1, 256)
(?, 1, 1, 256)


In [28]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 32, 32, 32)   864         input_7[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 32, 32, 32)   128         conv_1[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu_132 (LeakyReLU)     (None, 32, 32, 32)   0           norm_1[0][0]                     
__________________________________________________________________________________________________
max_poolin

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 7221s 144ms/step - loss: 1.8949 - acc: 0.2819 - val_loss: 1.5870 - val_acc: 0.4046
Epoch 2/30
50000/50000 [==============================] - 7198s 144ms/step - loss: 1.4050 - acc: 0.4814 - val_loss: 1.6891 - val_acc: 0.4370
Epoch 3/30
50000/50000 [==============================] - 7303s 146ms/step - loss: 1.1764 - acc: 0.5787 - val_loss: 1.4494 - val_acc: 0.5329
Epoch 4/30
50000/50000 [==============================] - 7394s 148ms/step - loss: 1.0077 - acc: 0.6429 - val_loss: 1.4990 - val_acc: 0.5483
Epoch 5/30
26368/50000 [==============>...............] - ETA: 58:18 - loss: 0.8921 - acc: 0.6862

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

In [0]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model.h5')